In [ ]:
#Optimizer 비교
from scratch.dataset.mnist import load_mnist
from scratch.common.util import smooth_curve
from scratch.common.multi_layer_net import MultiLayerNet
from scratch.common.optimizer import *
import matplotlib.pyplot as plt

# 0. MNIST 데이터 읽기==========
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True)

train_size = x_train.shape[0]
batch_size = 128
max_iterations = 1000

# 1. 실험용 설정==========
optimizers = {}
optimizers['SGD'] = SGD()
optimizers['Momentum'] = Momentum()
optimizers['AdaGrad'] = AdaGrad()
optimizers['Adam'] = Adam()
# optimizers['RMSprop'] = RMSprop()

networks = {}
train_loss = {}
for key in optimizers.keys():
    networks[key] = MultiLayerNet(
        input_size=784, hidden_size_list=[100, 100, 100, 100],
        output_size=10)
    train_loss[key] = []

# 2. 훈련 시작==========
for i in range(max_iterations):
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]

    for key in optimizers.keys():
        grads = networks[key].gradient(x_batch, t_batch)
        optimizers[key].update(networks[key].params, grads)

        loss = networks[key].loss(x_batch, t_batch)
        train_loss[key].append(loss)

    if i % 100 == 0:
        print("===========" + "iteration:" + str(i) + "===========")
        for key in optimizers.keys():
            loss = networks[key].loss(x_batch, t_batch)
            print(key + ":" + str(loss))

# 3. 그래프 그리기==========
markers = {"SGD": "o", "Momentum": "x", "AdaGrad": "s", "Adam": "D"}
x = np.arange(max_iterations)
plt.figure(figsize=(20, 8))
for key in optimizers.keys():
    plt.plot(x, smooth_curve(train_loss[key]), marker=markers[key], markevery=100, label=key)
plt.xlabel("iterations")
plt.ylabel("loss")
plt.ylim(0, 1)
plt.legend()
# plt.show()
# plt.savefig("a.png")


# SGD(Stochastic Gradient Descent) optimizer
Gradient descent 알고리즘은 시작 지점에서 기울기의 반대방향으로 하강하면서 loss function을 최소화 된 지점을 찾기 위한 가장 직관적인 방법입니다. 전체 데이터 셋을 가지고 학습 하게 되면 안정적이긴 하지만, 계산량과 학습 비용이 많아지게 됩니다.

이 때, 전체 data set이 아닌, data를 sampling해서 gradient descent를 진행하고, 이를 반복하며 정확도를 찾아나가는 것을 Stochastic Graident Descent라고 합니다.

http://cs231n.github.io/assets/nn3/nesterov.jpeg

Stochastic gradient descent는 수렴하는 동안 기울기가 진동하여 수렴 시간이 오래 걸리는 단점을 가지고 있습니다. 이를 보완하기 위해 momentum과 nestrov 기법이 사용되어 수렴을 좀 더 빠르게 할 수 있도록 도와줍니다.

실습을 통해서 optimization 기법 중 하나인 SGD 및 Nesterov momentum 을 확인해 봅시다.

참고자료 : http://aikorea.org/cs231n/neural-networks-3/#sgd

### 실습
Optimization 모델을 학습시킬 최적화 방법을 다음 optimizer를 사용하여 설정해보세요.
Stochastic Gradient Descent with nestrov

* optimizers.SGD(lr, momentum, nestrov) :
    * lr : learning rate (lr >= 0)
    * momentum : 진동을 막아주고 SGD를 가속화하는 파라메터(momentum >= 0)
    * nesterov : 미분 지점을 유연하게 변경하도록 돕는 파라메터(nesterov = True / False)
    * Refecence : https://keras.io/optimizers/

Default 값으로, lr = 0.1, momentum = 0.9, nesterov = True 값을 지정하여 실행시켜 봅시다.

https://kasausyrzlhe1066469.cdn.ntruss.com/global/file/p/5d3478902d4090f6c23d6f0f/sgd.png

learning rate, momentum, nestrov 매개변수를 변경해가면서 위의 그림과 같은 결과를 만들어보세요.

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import optimizers


# 각각 리뷰에 따른 데이터 길이가 다르기 때문에 데이터의 Shape을 맞춰줘야합니다.
def sequences_shaping(sequences, dimension):
    # 0으로 채워진 (len(sequences), dimension) 크기의 행렬을 만듭니다
    results = np.zeros((len(sequences), dimension))
    for i, word_indices in enumerate(sequences):
        results[i, word_indices] = 1.0  # 각 리뷰 별 빈도수가 높은 단어를 dimension 개수 만큼만 추출하여 사용합니다.
        
    return results

# 시각화 함수
def Visulaize(histories, key='binary_crossentropy'):
    #plt.figure(figsize=(,20))

    for name, history in histories:
        val = plt.plot(history.epoch, history.history['val_'+key],
                   '--', label=name.title()+' Val')
        plt.plot(history.epoch, history.history[key], color=val[0].get_color(),
             label=name.title()+' Train')

    plt.xlabel('Epochs')
    plt.ylabel(key.replace('_',' ').title())
    plt.legend()

    plt.xlim([0,max(history.epoch)])

#     plt.savefig("plot.png")

# 100번째 까지 많이 사용하는 단어까지 추출
word_num = 100
data_num = 25000

# Keras에 내장되어 있는 imdb 데이터 세트를 불러옵니다.
# IMDb 데이터 세트는 Train 25000개 test 25000개로 이루어져 있습니다.
(train_data, train_labels), (test_data, test_labels) = keras.datasets.imdb.load_data(num_words=word_num)

# 데이터 Shape을 맞춰주기 위한 sequence 함수를 불러옵니다.
train_data = sequences_shaping(train_data, dimension=word_num)
test_data = sequences_shaping(test_data, dimension=word_num)

# Optimizer 모델와 비교하기 위해 기본 모델을 하나 만들어줍니다.
basic_model = keras.Sequential([
    # `.summary` 메서드 때문에 `input_shape`가 필요합니다
    # 첫 번째 Layer에 데이터를 넣을때는 input_shape을 맞춰줘야합니다.
    keras.layers.Dense(128, activation=tf.nn.relu, input_shape=(word_num,)),
    keras.layers.Dense(128, activation=tf.nn.relu),
    keras.layers.Dense(1, activation=tf.nn.sigmoid)
])

# Optimizer를 적용할 모델을 생성합니다.
optimizer_model = keras.Sequential([
    keras.layers.Dense(128, activation=tf.nn.relu, input_shape=(word_num,)),
    keras.layers.Dense(128, activation=tf.nn.relu),
    keras.layers.Dense(1, activation=tf.nn.sigmoid)
])

# 기존 모델을 학습시킬 최적화 방법, loss 계산 방법, 평가 방법을 설정합니다.
basic_model.compile(optimizer='sgd',
                    loss='binary_crossentropy',
                    metrics=['accuracy', 'binary_crossentropy'])
# 현재 모델이 어떻게 이루어져있는지 출력합니다.
basic_model.summary()
# 모델을 학습시킵니다.
basic_history = basic_model.fit(train_data,train_labels,epochs=20,batch_size=500,validation_data=(test_data, test_labels), verbose=2)

'''
Optimization 모델을 학습시킬 최적화 방법, loss 계산 방법, 평가 방법을 설정해보세요.
'''
optimizer_model.compile(optimizer= optimizers.SGD(0.1, 0.9, True),
                        loss='binary_crossentropy',
                        metrics=['accuracy', 'binary_crossentropy'])
# Optimization 모델이 어떻게 이루어져있는지 출력합니다.
optimizer_model.summary()
# Optimization 모델을 학습시킵니다.
optimizer_history = optimizer_model.fit(train_data, train_labels, epochs=20, batch_size=500, validation_data=(test_data, test_labels),verbose=2)


# 각 모델 별 Loss 그래프를 그려줍니다.
Visulaize([('Basic', basic_history),('SGD with nestrov', optimizer_history)])
              


C:\Users\user\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


ModuleNotFoundError: No module named 'tensorflow.keras'

# Adagrad optimizer
Adagrad는 loss function에서 최소값을 가지는 최적의 파라메터 W(weights; 가중치)를 찾아내기 위해 step size를 조절해 하강하는 방법 중 하나입니다.

기존에 방식이 파라메터들의 학습속도를 같은 속도로 한꺼번에 갱신하는 방법이었다면, Adagrad는 파라메터 각각의 학습 속도를 데이터에 맞추어(adaptively) 계산하여 적절한 step size로 하강하도록 합니다.

실제로 Adagrad를 적용하면 높은 그라디언트값을 갖는 파라메터(weight)들은 점점 학습속도가 감소하게 되고, 그라디언트 값이 낮거나 업데이트가 거의 없는 파라메터들은 실질 학습속도가 증가하게 됩니다. 이를 통해 최소값에 대한 효율적인 탐색이 가능해집니다.

실습을 통해 Adagrad optimization 에 대해 좀 더 알아봅시다.

참고자료 : https://keras.io/optimizers/

### 실습
Optimization 모델을 학습시킬 최적화 방법을 다음 optimizer를 사용하여 설정해보세요.
Adagrad optimizer

* optimizers.Adagrad(lr, epsilon, decay) :
    * lr : learning rate (lr >= 0)
    * epsilon : 연산 시 분모가 0이 되는 것을 막는 0에 가까운 상수(epsilon >= 0; None = default K.epsilon())
    * decay : 각 업데이트 시 learning rate를 비율 만큼 줄여 주는 파라메터(decay >= 0)
    * Refecence : https://keras.io/optimizers/

Default 값으로, lr = 0.1, epsilon = None, decay = 0.0 값을 지정하여 실행시켜 봅시다.

https://kasausyrzlhe1066469.cdn.ntruss.com/global/file/p/5d3481fb14bf3ed0abc8dfef/adagrad.png

learning rate, epsilon, decay 매개변수를 변경해가면서 위의 그림과 같은 결과를 만들어보세요.

In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import optimizers

# 각각 리뷰에 따른 데이터 길이가 다르기 때문에 데이터의 Shape을 맞춰줘야합니다.
def sequences_shaping(sequences, dimension):
    # 0으로 채워진 (len(sequences), dimension) 크기의 행렬을 만듭니다
    results = np.zeros((len(sequences), dimension))
    for i, word_indices in enumerate(sequences):
        results[i, word_indices] = 1.0  # 각 리뷰 별 빈도수가 높은 단어를 dimension 개수 만큼만 추출하여 사용합니다.
        
    return results

# 시각화 함수
def Visulaize(histories, key='binary_crossentropy'):
    #plt.figure(figsize=(,20))

    for name, history in histories:
        val = plt.plot(history.epoch, history.history['val_'+key],
                   '--', label=name.title()+' Val')
        plt.plot(history.epoch, history.history[key], color=val[0].get_color(),
             label=name.title()+' Train')

    plt.xlabel('Epochs')
    plt.ylabel(key.replace('_',' ').title())
    plt.legend()

    plt.xlim([0,max(history.epoch)])

#     plt.savefig("plot.png")

# 100번째 까지 많이 사용하는 단어까지 추출
word_num = 100
data_num = 25000

# Keras에 내장되어 있는 imdb 데이터 세트를 불러옵니다.
# IMDb 데이터 세트는 Train 25000개 test 25000개로 이루어져 있습니다.
(train_data, train_labels), (test_data, test_labels) = keras.datasets.imdb.load_data(num_words=word_num)

# 데이터 Shape을 맞춰주기 위한 sequence 함수를 불러옵니다.
train_data = sequences_shaping(train_data, dimension=word_num)
test_data = sequences_shaping(test_data, dimension=word_num)

# Optimizer 모델와 비교하기 위해 기본 모델을 하나 만들어줍니다.
basic_model = keras.Sequential([
    # `.summary` 메서드 때문에 `input_shape`가 필요합니다
    # 첫 번째 Layer에 데이터를 넣을때는 input_shape을 맞춰줘야합니다.
    keras.layers.Dense(128, activation=tf.nn.relu, input_shape=(word_num,)),
    keras.layers.Dense(128, activation=tf.nn.relu),
    keras.layers.Dense(1, activation=tf.nn.sigmoid)
])

# Optimizer를 적용할 모델을 생성합니다.
optimizer_model = keras.Sequential([
    keras.layers.Dense(128, activation=tf.nn.relu, input_shape=(word_num,)),
    keras.layers.Dense(128, activation=tf.nn.relu),
    keras.layers.Dense(1, activation=tf.nn.sigmoid)
])

# 기존 모델을 학습시킬 최적화 방법, loss 계산 방법, 평가 방법을 설정합니다.
basic_model.compile(optimizer='sgd',
                    loss='binary_crossentropy',
                    metrics=['accuracy', 'binary_crossentropy'])
# 현재 모델이 어떻게 이루어져있는지 출력합니다.
basic_model.summary()
# 모델을 학습시킵니다.
basic_history = basic_model.fit(train_data,train_labels,epochs=20,batch_size=500,validation_data=(test_data, test_labels), verbose=2)

'''
Optimization 모델을 학습시킬 최적화 방법, loss 계산 방법, 평가 방법을 설정해보세요.
'''
optimizer_model.compile(optimizer = optimizers.Adagrad(lr = 0.01, decay = 0.01, epsilon = None),
                        loss='binary_crossentropy',
                        metrics=['accuracy', 'binary_crossentropy'])
# Optimization 모델이 어떻게 이루어져있는지 출력합니다.
optimizer_model.summary()
# Optimization 모델을 학습시킵니다.
optimizer_history = optimizer_model.fit(train_data, train_labels, epochs=20, batch_size=500, validation_data=(test_data, test_labels),verbose=2)


# 각 모델 별 Loss 그래프를 그려줍니다.
Visulaize([('Basic', basic_history),('Adagrad', optimizer_history)])
              


ModuleNotFoundError: No module named 'tensorflow.keras'

# Adadelta optimizer
Adadelta(Adaptive Delta) 는 Adagrad의 단점을 보완하기 위해 만들어진 방법입니다.

보통 adagrad에서는 0.01 정도를 learning rate로 지정하게 되면, 다른 설정값(hyper-parameter) 없이 학습이 잘 진행됩니다. 따라서 설정값 여러 개에 크게 의존하지 않는다는 장점이 있습니다 . 하지만, 학습이 오래 진행될 경우 step size가 너무 작아져서 결국 거의 움직이지 않게 됩니다.

Adadelta는 2차 최적화(2nd order optimization) 기법을 따와서 학습 진행에 따라 0에 가까워지는 step size를 조절해 나갈 수 있도록 개선된 알고리즘입니다.

실습을 통해서 optimization 기법 중 하나인 Adadelta 알고리즘을 확인해 봅시다.

### 실습
Optimization 모델을 학습시킬 최적화 방법을 다음 optimizer를 사용하여 설정해보세요.
### Adadelta optimizer

* optimizers.Adadelta(lr, rho, epsilon, decay) :
    * lr : 초기 learning rate (lr >= 0)
    * rho : Adadelta decay factor (rho >= 0)
    * epsilon : 연산 시 분모가 0이 되는 것을 막는 0에 가까운 상수(epsilon >= 0; None = default K.epsilon())
    * decay : 각 업데이트 시 learning rate를 비율 만큼 줄여 주는 파라메터(decay >= 0)
    * Refecence : https://keras.io/optimizers/

Default 값으로, lr = 0.1, rho = 0.95, epsilon = None, decay = 0.5 값을 지정하여 실행시켜 봅시다.


https://kasausyrzlhe1066469.cdn.ntruss.com/global/file/p/5d348b1280f665f8bb30a1ae/adadelta.png

learning rate, rho, epsilon, decay 매개변수를 변경해가면서 위의 그림과 같은 결과를 만들어보세요.

In [4]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import optimizers

# 각각 리뷰에 따른 데이터 길이가 다르기 때문에 데이터의 Shape을 맞춰줘야합니다.
def sequences_shaping(sequences, dimension):
    # 0으로 채워진 (len(sequences), dimension) 크기의 행렬을 만듭니다
    results = np.zeros((len(sequences), dimension))
    for i, word_indices in enumerate(sequences):
        results[i, word_indices] = 1.0  # 각 리뷰 별 빈도수가 높은 단어를 dimension 개수 만큼만 추출하여 사용합니다.
        
    return results

# 시각화 함수
def Visulaize(histories, key='binary_crossentropy'):
    #plt.figure(figsize=(,20))

    for name, history in histories:
        val = plt.plot(history.epoch, history.history['val_'+key],
                   '--', label=name.title()+' Val')
        plt.plot(history.epoch, history.history[key], color=val[0].get_color(),
             label=name.title()+' Train')

    plt.xlabel('Epochs')
    plt.ylabel(key.replace('_',' ').title())
    plt.legend()

    plt.xlim([0,max(history.epoch)])

#     plt.savefig("plot.png")

# 100번째 까지 많이 사용하는 단어까지 추출
word_num = 100
data_num = 25000

# Keras에 내장되어 있는 imdb 데이터 세트를 불러옵니다.
# IMDb 데이터 세트는 Train 25000개 test 25000개로 이루어져 있습니다.
(train_data, train_labels), (test_data, test_labels) = keras.datasets.imdb.load_data(num_words=word_num)

# 데이터 Shape을 맞춰주기 위한 sequence 함수를 불러옵니다.
train_data = sequences_shaping(train_data, dimension=word_num)
test_data = sequences_shaping(test_data, dimension=word_num)

# Optimizer 모델와 비교하기 위해 기본 모델을 하나 만들어줍니다.
basic_model = keras.Sequential([
    # `.summary` 메서드 때문에 `input_shape`가 필요합니다
    # 첫 번째 Layer에 데이터를 넣을때는 input_shape을 맞춰줘야합니다.
    keras.layers.Dense(128, activation=tf.nn.relu, input_shape=(word_num,)),
    keras.layers.Dense(128, activation=tf.nn.relu),
    keras.layers.Dense(1, activation=tf.nn.sigmoid)
])

# Optimizer를 적용할 모델을 생성합니다.
optimizer_model = keras.Sequential([
    keras.layers.Dense(128, activation=tf.nn.relu, input_shape=(word_num,)),
    keras.layers.Dense(128, activation=tf.nn.relu),
    keras.layers.Dense(1, activation=tf.nn.sigmoid)
])

# 기존 모델을 학습시킬 최적화 방법, loss 계산 방법, 평가 방법을 설정합니다.
basic_model.compile(optimizer='sgd',
                    loss='binary_crossentropy',
                    metrics=['accuracy', 'binary_crossentropy'])
# 현재 모델이 어떻게 이루어져있는지 출력합니다.
basic_model.summary()
# 모델을 학습시킵니다.
basic_history = basic_model.fit(train_data,train_labels,epochs=20,batch_size=500,validation_data=(test_data, test_labels), verbose=2)

'''
Optimization 모델을 학습시킬 최적화 방법, loss 계산 방법, 평가 방법을 설정해보세요.
'''
optimizer_model.compile(optimizer= optimizers.Adadelta(0.05, 0.80, 0.4, None),
                        loss='binary_crossentropy',
                        metrics=['accuracy', 'binary_crossentropy']) 
# Optimization 모델이 어떻게 이루어져있는지 출력합니다.
optimizer_model.summary()
# Optimization 모델을 학습시킵니다.
optimizer_history = optimizer_model.fit(train_data, train_labels, epochs=20, batch_size=500, validation_data=(test_data, test_labels),verbose=2)


# 각 모델 별 Loss 그래프를 그려줍니다.
Visulaize([('Basic', basic_history),('Adadelta', optimizer_history)])
              


ModuleNotFoundError: No module named 'tensorflow.keras'

# RMSprop optimizer
RMSprop 또한 Adagrad 기법의 단점을 보완하고자 제안된 것으로, 딥러닝의 대가 제프리 힌튼 교수가 처음 언급한 방법입니다.

RMSProp 업데이트는 Adagrad 기법에서 각 미분 값의 제곱을 그대로 더하지 않고, decay 파라메터로 조정하여 빠르게 감소하는 학습속도를 어느 정도 경감시키는 역할을 합니다. 이는 Adagrad에서의 제곱 그라디언트의 평균을 사용하지 않고, 이동평균(moving average)을 사용한 방법입니다.

구체적인 실습을 통해 RMSprop optimizer에 대해 확인해보도록 합시다.

참고자료 : http://aikorea.org/cs231n/neural-networks-3/#ada

### 실습
Optimization 모델을 학습시킬 최적화 방법을 다음 optimizer를 사용하여 설정해보세요.
#### RMSprop optimizer

* optimizers.RMSprop(lr) :
    * lr : learning rate ( lr >= 0 )
    * Refecence : https://keras.io/optimizers/

Default 값으로, lr = 0.001 값을 지정하여 실행시켜 봅시다.

https://kasausyrzlhe1066469.cdn.ntruss.com/global/file/p/5d34912303c0972ab6d2dc92/rmsprop.png

learning rate 매개변수를 변경해가면서 위의 그림과 같은 결과를 만들어보세요.

In [5]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import optimizers

# 각각 리뷰에 따른 데이터 길이가 다르기 때문에 데이터의 Shape을 맞춰줘야합니다.
def sequences_shaping(sequences, dimension):
    # 0으로 채워진 (len(sequences), dimension) 크기의 행렬을 만듭니다
    results = np.zeros((len(sequences), dimension))
    for i, word_indices in enumerate(sequences):
        results[i, word_indices] = 1.0  # 각 리뷰 별 빈도수가 높은 단어를 dimension 개수 만큼만 추출하여 사용합니다.
        
    return results

# 시각화 함수
def Visulaize(histories, key='binary_crossentropy'):
    #plt.figure(figsize=(,20))

    for name, history in histories:
        val = plt.plot(history.epoch, history.history['val_'+key],
                   '--', label=name.title()+' Val')
        plt.plot(history.epoch, history.history[key], color=val[0].get_color(),
             label=name.title()+' Train')

    plt.xlabel('Epochs')
    plt.ylabel(key.replace('_',' ').title())
    plt.legend()

    plt.xlim([0,max(history.epoch)])

#     plt.savefig("plot.png")

# 100번째 까지 많이 사용하는 단어까지 추출
word_num = 100
data_num = 25000

# Keras에 내장되어 있는 imdb 데이터 세트를 불러옵니다.
# IMDb 데이터 세트는 Train 25000개 test 25000개로 이루어져 있습니다.
(train_data, train_labels), (test_data, test_labels) = keras.datasets.imdb.load_data(num_words=word_num)

# 데이터 Shape을 맞춰주기 위한 sequence 함수를 불러옵니다.
train_data = sequences_shaping(train_data, dimension=word_num)
test_data = sequences_shaping(test_data, dimension=word_num)

# Optimizer 모델와 비교하기 위해 기본 모델을 하나 만들어줍니다.
basic_model = keras.Sequential([
    # `.summary` 메서드 때문에 `input_shape`가 필요합니다
    # 첫 번째 Layer에 데이터를 넣을때는 input_shape을 맞춰줘야합니다.
    keras.layers.Dense(128, activation=tf.nn.relu, input_shape=(word_num,)),
    keras.layers.Dense(128, activation=tf.nn.relu),
    keras.layers.Dense(1, activation=tf.nn.sigmoid)
])

# Optimizer를 적용할 모델을 생성합니다.
optimizer_model = keras.Sequential([
    keras.layers.Dense(128, activation=tf.nn.relu, input_shape=(word_num,)),
    keras.layers.Dense(128, activation=tf.nn.relu),
    keras.layers.Dense(1, activation=tf.nn.sigmoid)
])

# 기존 모델을 학습시킬 최적화 방법, loss 계산 방법, 평가 방법을 설정합니다.
basic_model.compile(optimizer='sgd',
                    loss='binary_crossentropy',
                    metrics=['accuracy', 'binary_crossentropy'])
# 현재 모델이 어떻게 이루어져있는지 출력합니다.
basic_model.summary()
# 모델을 학습시킵니다.
basic_history = basic_model.fit(train_data,train_labels,epochs=20,batch_size=500,validation_data=(test_data, test_labels), verbose=2)

'''
Optimization 모델을 학습시킬 최적화 방법, loss 계산 방법, 평가 방법을 설정해보세요.
'''
optimizer_model.compile(optimizer= optimizers.RMSprop(0.0001),
                        loss='binary_crossentropy',
                        metrics=['accuracy', 'binary_crossentropy'])
# Optimization 모델이 어떻게 이루어져있는지 출력합니다.
optimizer_model.summary()
# Optimization 모델을 학습시킵니다.
optimizer_history = optimizer_model.fit(train_data, train_labels, epochs=20, batch_size=500, validation_data=(test_data, test_labels),verbose=2)


# 각 모델 별 Loss 그래프를 그려줍니다.
Visulaize([('Basic', basic_history),('RMSprop', optimizer_history)])
              


ModuleNotFoundError: No module named 'tensorflow.keras'

# Adam optimizer
http://i.imgur.com/NKsFHJb.gif?1

Adam optimizer는 최적화 기법 중 가장 발전된 기법입니다. RMSProp 기법과 모멘텀(momentum)을 함께 사용함으로써, 진행 방향과 step size 모두 적절하게 유지하면서 학습할 수 있도록 고안되었습니다.

실습을 통해 Adam optimizer에 대해 좀 더 알아보도록 하겠습니다.

참고자료 : http://shuuki4.github.io/deep%20learning/2016/05/20/Gradient-Descent-Algorithm-Overview.html

### 실습
Optimization 모델을 학습시킬 최적화 방법을 다음 optimizer를 사용하여 설정해보세요.
#### Adam optimizer

* optimizers.Adam(lr, beta_1, beta_2) :
    * lr : learning rate ( lr >= 0 )
    * beta_1 : momentum을 결정하기 위해 사용되는 파라메터(beta_1 >= 0 )
    * beta_2 : step size를 결정하기 위해 사용되는 파라메터(beta_2 >= 0)
    * Refecence : https://keras.io/optimizers/

Default 값으로, lr = 0.01, beta_1 = 0.9, beta_2 = 0.999 값을 지정하여 실행시켜 봅시다.

https://kasausyrzlhe1066469.cdn.ntruss.com/global/file/p/5d34946880f665f8bb30a1b6/adam.png

learning rate, beta_1, beta_2 매개변수를 변경해가면서 위의 그림과 같은 결과를 만들어보세요.

In [6]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import optimizers

# 각각 리뷰에 따른 데이터 길이가 다르기 때문에 데이터의 Shape을 맞춰줘야합니다.
def sequences_shaping(sequences, dimension):
    # 0으로 채워진 (len(sequences), dimension) 크기의 행렬을 만듭니다
    results = np.zeros((len(sequences), dimension))
    for i, word_indices in enumerate(sequences):
        results[i, word_indices] = 1.0  # 각 리뷰 별 빈도수가 높은 단어를 dimension 개수 만큼만 추출하여 사용합니다.
        
    return results

# 시각화 함수
def Visulaize(histories, key='binary_crossentropy'):
    #plt.figure(figsize=(,20))

    for name, history in histories:
        val = plt.plot(history.epoch, history.history['val_'+key],
                   '--', label=name.title()+' Val')
        plt.plot(history.epoch, history.history[key], color=val[0].get_color(),
             label=name.title()+' Train')

    plt.xlabel('Epochs')
    plt.ylabel(key.replace('_',' ').title())
    plt.legend()

    plt.xlim([0,max(history.epoch)])

#     plt.savefig("plot.png")

# 100번째 까지 많이 사용하는 단어까지 추출
word_num = 100
data_num = 25000

# Keras에 내장되어 있는 imdb 데이터 세트를 불러옵니다.
# IMDb 데이터 세트는 Train 25000개 test 25000개로 이루어져 있습니다.
(train_data, train_labels), (test_data, test_labels) = keras.datasets.imdb.load_data(num_words=word_num)

# 데이터 Shape을 맞춰주기 위한 sequence 함수를 불러옵니다.
train_data = sequences_shaping(train_data, dimension=word_num)
test_data = sequences_shaping(test_data, dimension=word_num)

# Optimizer 모델와 비교하기 위해 기본 모델을 하나 만들어줍니다.
basic_model = keras.Sequential([
    # `.summary` 메서드 때문에 `input_shape`가 필요합니다
    # 첫 번째 Layer에 데이터를 넣을때는 input_shape을 맞춰줘야합니다.
    keras.layers.Dense(128, activation=tf.nn.relu, input_shape=(word_num,)),
    keras.layers.Dense(128, activation=tf.nn.relu),
    keras.layers.Dense(1, activation=tf.nn.sigmoid)
])

# Optimizer를 적용할 모델을 생성합니다.
optimizer_model = keras.Sequential([
    keras.layers.Dense(128, activation=tf.nn.relu, input_shape=(word_num,)),
    keras.layers.Dense(128, activation=tf.nn.relu),
    keras.layers.Dense(1, activation=tf.nn.sigmoid)
])

# 기존 모델을 학습시킬 최적화 방법, loss 계산 방법, 평가 방법을 설정합니다.
basic_model.compile(optimizer='sgd',
                    loss='binary_crossentropy',
                    metrics=['accuracy', 'binary_crossentropy'])
# 현재 모델이 어떻게 이루어져있는지 출력합니다.
basic_model.summary()
# 모델을 학습시킵니다.
basic_history = basic_model.fit(train_data,train_labels,epochs=20,batch_size=500,validation_data=(test_data, test_labels), verbose=2)

'''
Optimization 모델을 학습시킬 최적화 방법, loss 계산 방법, 평가 방법을 설정해보세요.
'''
optimizer_model.compile(optimizer= optimizers.Adam(0.0001, 0.8, 0.8),
                        loss='binary_crossentropy',
                        metrics=['accuracy', 'binary_crossentropy'])
# Optimization 모델이 어떻게 이루어져있는지 출력합니다.
optimizer_model.summary()
# Optimization 모델을 학습시킵니다.
optimizer_history = optimizer_model.fit(train_data, train_labels, epochs=20, batch_size=500, validation_data=(test_data, test_labels),verbose=2)


# 각 모델 별 Loss 그래프를 그려줍니다.
Visulaize([('Basic', basic_history),('Adam', optimizer_history)])
              


ModuleNotFoundError: No module named 'tensorflow.keras'

# Optimizer로 성능 높여보기

http://blog.datumbox.com/wp-content/uploads/2013/10/gradient-descent.png
    
(사진 출처 : AI Society)

지금까지 배운 optimizer를 활용하여 다양한 도메인에서 활용해봅시다.

Fashion-mnist data를 가지고 인공신경망을 활용해서 loss function인 'sparse categorical crossentropy’를 가장 최소화함으로써 분류 문제를 해결하려고 합니다.

이번 미션을 통해서 직접 최적화 된 인공신경망을 구현해 보세요.

### 실습
기본 모델을 참고하여 Optimizer를 적용할 모델을 생성해보세요.

기본 모델을 참고하여 Optimizer 모델의 optimizer, loss, metrics를 입력해보세요.

기본 모델을 참고하여 Optimzer 모델로 테스트 데이터를 예측하는 코드를 작성해 보세요.

https://kasausyrzlhe1066469.cdn.ntruss.com/global/file/p/5d357e66230dcc777a8aeb28/mission.png

Optimizer 매개변수를 변경해가면서 기본 모델보다 cross-entropy 결과가 더 낮은 모델을 만들어보세요.

In [7]:
# tensorflow와 tf.keras를 임포트합니다
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import optimizers
import matplotlib.pyplot as plt


def Visulaize(histories, key='sparse_categorical_crossentropy'):

    for name, history in histories:
        val = plt.plot(history.epoch, history.history['val_'+key],
                   '--', label=name.title()+' Val')
        plt.plot(history.epoch, history.history[key], color=val[0].get_color(),
             label=name.title()+' Train')

    plt.xlabel('Epochs')
    plt.ylabel(key.replace('_',' ').title())
    plt.legend()

    plt.xlim([0,max(history.epoch)])
    
#     plt.savefig("plot.png")    

# Fashion mnist data 를 load 합니다.
train_images = np.loadtxt('./data/train_images.csv', delimiter =',', dtype = np.float32)
train_labels = np.loadtxt('./data/train_labels.csv', delimiter =',', dtype = np.float32)
test_images = np.loadtxt('./data/test_images.csv', delimiter =',', dtype = np.float32)
test_labels = np.loadtxt('./data/test_labels.csv', delimiter =',', dtype = np.float32)

train_images = train_images / 255.0
test_images = test_images / 255.0

# label에 해당하는 class name 입니다.
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']


# 2-layer 기본 모델을 생성합니다.
model = keras.Sequential([
    # 첫번째 layer의 뉴런 개수를 정합니다.
    keras.layers.Dense(128, activation=tf.nn.relu),
    # 두번째 layer의 뉴런 개수를 정합니다. (output)
    keras.layers.Dense(10, activation=tf.nn.softmax)
])

# 1. 기본 모델을 참고하여 Optimizer를 적용할 모델을 생성해보세요.
optimizer_model = keras.Sequential([
    # 첫번째 layer의 뉴런 개수를 정합니다.
    keras.layers.Dense(128, activation=tf.nn.relu),
    # 두번째 layer의 뉴런 개수를 정합니다. (output)
    keras.layers.Dense(10, activation=tf.nn.softmax)
])

# 기본 모델의 상세 기법을 정의합니다.
model.compile(optimizer='sgd',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy','sparse_categorical_crossentropy'])

# 2. 기본 모델을 참고하여 Optimizer 모델의 optimizer, loss, metrics를 입력해보세요.
optimizer_model.compile(optimizer= optimizers.Adam(0.001, 0.8, 0.8),
                        loss='sparse_categorical_crossentropy',
                        metrics=['accuracy', 'sparse_categorical_crossentropy'])


# 학습한 기본 모델로 테스트 데이터를 예측합니다.
basic_history = model.fit(train_images, train_labels, epochs=20, batch_size=500, validation_data=(test_images, test_labels),verbose=2)

# 3. 기본 모델을 참고하여 Optimzer 모델로 테스트 데이터를 예측하는 코드를 작성해 보세요.
optimizer_history = optimizer_model.fit(train_images, train_labels, epochs=20, batch_size=500, validation_data=(test_images, test_labels),verbose=2)


basic_score = model.evaluate(test_images, test_labels)
optimizer_score = optimizer_model.evaluate(test_images, test_labels)


# 각 모델 별 Loss 그래프를 그려줍니다.
Visulaize([('Basic', basic_history),('Adam', optimizer_history)])

ModuleNotFoundError: No module named 'tensorflow.keras'